In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import time
import re
from urllib.parse import urljoin

class ToVimaScraperException(Exception):
    """Custom exception for scraper errors"""
    pass

class ToVimaScraper:
    def __init__(self, base_url, output_dir):
        """
        Initialize the scraper with base URL and output directory
        
        Args:
            base_url (str): The starting URL for scraping
            output_dir (str): Directory to save the scraped articles
        """
        self.base_url = base_url
        self.output_dir = output_dir
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        self.create_output_directory()

    def create_output_directory(self):
        """Create the output directory if it doesn't exist"""
        if not os.path.exists(self.output_dir):
            os.makedirs(self.output_dir)

    def get_page_content(self, url):
        """
        Fetch the content of a webpage
        
        Args:
            url (str): URL to fetch
            
        Returns:
            BeautifulSoup object or None if failed
        """
        try:
            response = requests.get(url, headers=self.headers)
            response.raise_for_status()
            return BeautifulSoup(response.text, 'html.parser')
        except requests.RequestException as e:
            print(f"Error fetching {url}: {str(e)}")
            return None

    def extract_article_links(self, soup):
        """
        Extract article links and titles from the page
        
        Args:
            soup (BeautifulSoup): Parsed HTML of the page
            
        Returns:
            list: List of tuples containing (title, url)
        """
        articles = []
        for link in soup.find_all('a', {'class': 'columns is-mobile is-multiline'}):
            title = link.get('title')
            url = link.get('href')
            if title and url:
                articles.append((title, url))
        return articles

    def get_next_page_url(self, soup):
        """
        Find the URL of the next page
        
        Args:
            soup (BeautifulSoup): Parsed HTML of the current page
            
        Returns:
            str or None: URL of the next page or None if not found
        """
        next_link = soup.find('a', title='Επόμενη Σελίδα')
        return next_link.get('href') if next_link else None

    def extract_article_text(self, soup):
        """
        Extract the plain text content from an article
        
        Args:
            soup (BeautifulSoup): Parsed HTML of the article
            
        Returns:
            str: Plain text content of the article
        """
        article_div = soup.find('div', {'class': 'post-body main-content pos-rel article-wrapper'})
        if not article_div:
            return None

        # Remove unwanted elements
        for element in article_div.find_all(['script', 'style', 'iframe', 'div', 'figure']):
            element.decompose()

        # Extract text from paragraphs
        paragraphs = article_div.find_all('p')
        text = '\n\n'.join(p.get_text().strip() for p in paragraphs)
        
        # Clean up the text
        text = re.sub(r'\n{3,}', '\n\n', text)  # Remove excessive newlines
        text = re.sub(r'\s+', ' ', text)  # Remove excessive whitespace
        
        return text.strip()

    def save_article(self, title, content, index):
        """
        Save article content to a file
        
        Args:
            title (str): Article title
            content (str): Article content
            index (int): Article index for filename
        """
        if not content:
            return

        # Create a safe filename from the title
        safe_title = re.sub(r'[^\w\s-]', '', title)
        safe_title = re.sub(r'\s+', '_', safe_title)
        filename = f"{index:03d}_{safe_title[:50]}.txt"
        
        filepath = os.path.join(self.output_dir, filename)
        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(f"Title: {title}\n\n")
            f.write(content)

    def scrape_articles(self, max_articles=100):
        """
        Main method to scrape articles
        
        Args:
            max_articles (int): Maximum number of articles to scrape
        """
        current_url = self.base_url
        article_count = 0
        
        while current_url and article_count < max_articles:
            print(f"Processing page: {current_url}")
            
            # Get the page content
            soup = self.get_page_content(current_url)
            if not soup:
                break

            # Extract article links
            articles = self.extract_article_links(soup)
            
            # Process each article
            for title, article_url in articles:
                if article_count >= max_articles:
                    break
                    
                print(f"Scraping article {article_count + 1}: {title}")
                
                # Get article content
                article_soup = self.get_page_content(article_url)
                if article_soup:
                    content = self.extract_article_text(article_soup)
                    if content:
                        self.save_article(title, content, article_count)
                        article_count += 1
                
                # Be nice to the server
                time.sleep(1)
            
            # Get next page URL
            current_url = self.get_next_page_url(soup)
            
            # Be nice to the server between pages
            time.sleep(2)

        print(f"Finished scraping {article_count} articles")

# Usage example
if __name__ == "__main__":
    BASE_URL = "https://www.tovima.gr/category/politics/"
    OUTPUT_DIR = "D:\\Web Scrapping Project\\tovima_articles"
    
    scraper = ToVimaScraper(BASE_URL, OUTPUT_DIR)
    scraper.scrape_articles(max_articles=100)

In [ ]:
'''
The output of the code is:

Processing page: https://www.tovima.gr/category/politics/
Scraping article 1: ΣΥΡΙΖΑ – Κασσελάκης: Στα μαχαίρια για συνέδριο, νοθεία, μητρώο, δεοντολογία και ΟΜ
Scraping article 2: Κακλαμάνης για Μητσοτάκη – Σαμαρά: Σηκώνεις το τηλέφωνο και λες «έλα να το λύσουμε»
Scraping article 3: ΣΥΡΙΖΑ: Ο Μητσοτάκης δεν μπορεί να ξεγλιστρήσει από τη χυδαιότητα
Scraping article 4: Τζάκρη: Ο Κασσελάκης δεν έδιωξε και δεν θα διώξει κανέναν
Scraping article 5: Βουλή: Οσα έγιναν χθες – Σε τρία μέτωπα η επίθεση Μητσοτάκη
Scraping article 6: Εκλογές ΣΥΡΙΖΑ: Λήξη διορίας – Ούτε βήμα πίσω ο Κασσελάκης
Scraping article 7: Στέφανος Κασσελάκης: Άνοιξαν οι πόρτες των γραφείων του – Καπνογόνα στην άφιξη
Scraping article 8: ΣΥΡΙΖΑ κατά Μητσοτάκη: «Εκτροπή και ύβρη του πρωθυπουργού των σκανδάλων»
Scraping article 9: Αγριεύει η κόντρα Μητσοτάκη-Σαμαρά: «Να ασχοληθούμε με τα σοβαρά στα σύνορά μας»
Scraping article 10: Γαλάζιες αντιθέσεις απέναντι σε Παππά – Ο Μητσοτάκης έφυγε, ο Σαμαράς χαιρέτησε
Scraping article 11: Νίκος Παππάς: Η απάντηση στην κίνηση Μητσοτάκη – Κακομαθημένο παιδί
Scraping article 12: Ανδρουλάκης σε Μητσοτάκη: «Οι μέρες της παντοδυναμίας σας ανήκουν στο παρελθόν»
Processing page: https://www.tovima.gr/category/politics/page/2/
Scraping article 13: Παππάς στη Βουλή για τις φωτιές: Τι από αυτά είναι προϊόν φαντασίας;
Scraping article 14: Βουλή: Επίθεση Μητσοτάκη σε Παππά – «Δεν αναγνωρίζω τον 13-0» – Πυρά και κατά Ανδρουλάκη
Scraping article 15: Μητσοτάκης: «Ήταν η πιο δύσκολη αντιπυρική περίοδος των τελευταίων 40 ετών»
Scraping article 16: Βουλή Live – Μητσοτάκης: Τo 112 σώζει ζωές – Οι ομιλίες στη Βουλή για τις φωτιές
Scraping article 17: ΠαΣοΚ: Πώς ο Μάντζος έσωσε τη Λιακούλη
Scraping article 18: Οι έμμεσες οδηγίες Τσίπρα για νέο κόμμα
Scraping article 19: ΠαΣοΚ: Τι ζήτησε ο Ανδρουλάκης στη συνεδρίαση της «Στρογγυλής Τραπέζης»
Scraping article 20: Σύγκρουση Μητσοτάκη-Ανδρουλάκη στη Βουλή: Οι κόντρες στο παρελθόν και οι φαρμακερές ατάκες
Scraping article 21: Τσίπρας: Χτυπάω καμπανάκι – Η χώρα χρειάζεται μεγάλο αναπτυξιακό σοκ
Scraping article 22: Ολοκληρώθηκε η ενημέρωση του Γεραπετρίτη στους γαλάζιους βουλευτές
Scraping article 23: Κυριάκος Μητσοτάκης σε Αντόνιο Κόστα: «Σε θεωρούμε έναν από εμάς»
Scraping article 24: Η Λιακούλη έδωσε τη σκυτάλη στον Μπιάγκη – Γεύμα για δύο στη Βουλή
Scraping article 25: Ο Γεραπετρίτης ενημερώνει τους γαλάζιους βουλευτές – Τα γλυκά που κέρασε η Ντόρα
Scraping article 26: Πρώτη του ΠαΣοΚ χωρίς Γερουλάνο – «Αιχμές» Άννας για τον Δούκα
Scraping article 27: Κασσελάκης: Παράτυπη η ψήφος Βερναρδάκη – Να δώσει εξηγήσεις η Σβίγκου
Scraping article 28: Νέα Δημοκρατία: Η απάντηση για το πρόστιμο των 40.000 ευρώ
Scraping article 29: Βουλή: Ο Λιβάνιος δεν δέχθηκε την τροπολογία του ΠαΣοΚ – Σύγκρουση Στίγκα με Δουδωνή
Scraping article 30: Φάμελλος: Στήριξη από Ιωακειμίδη – «Με τον Σωκράτη, αυτό θα κάνουμε σλόγκαν»
Scraping article 31: Πρόστιμο στη ΝΔ για τα αρχεία – Αφορμή τα Ασημακοπούλου Leaks
Scraping article 32: Εκλογές ΣΥΡΙΖΑ: Ο Κασσελάκης ανοίγει τα νέα γραφεία του στο κοινό
Processing page: https://www.tovima.gr/category/politics/page/3/
Scraping article 33: ΟΠΕΚΕΠΕ: Το λάθος, τα 50 εκ ευρώ και το εκρηκτικό κλίμα στην ύπαιθρο
Scraping article 34: Συνέδριο ΣΥΡΙΖΑ: Ο «πόλεμος» της 25ης Οκτωβρίου – Πώς επηρεάζεται ο Κασσελάκης
Scraping article 35: «Αφωνος» για 10 μέρες ο Παύλος Γερουλάνος – Ο λόγος
Scraping article 36: ΠαΣοΚ: Οι ενισχυμένοι των ανακοινώσεων Ανδρουλάκη -Γκρίνιες κι αντιδράσεις 
Scraping article 37: Ινστιτούτο Αλέξη Τσίπρα: Η εκδήλωση και η παρέμβαση του πρώην πρωθυπουργού
Scraping article 38: Γιατί «σφάζονται» η Λατινοπούλου με τον Βελόπουλο – Ο πόλεμος στα δεξιά της ΝΔ
Scraping article 39: ΠαΣοΚ: Οι τρεις γυναίκες που ξάφνιασαν – Το προφίλ των Πλέσια, Σαμαρά και Σολωμού
Scraping article 40: Ελεονώρα Μελέτη σε ΕΕ: Οι κίνδυνοι των παιδιών στο διαδίκτυο
Scraping article 41: Η στρατηγική της ΝΔ για Λατινοπούλου, Βελόπουλο
Scraping article 42: Δημοσκόπηση ALCO: Άνοδος για το ΠαΣοΚ – Συνεχίζεται η κατηφόρα του ΣΥΡΙΖΑ
Scraping article 43: Υποκλοπές: Ευρωπαϊκή Επιτροπή και ΕΔΔΑ προστατεύουν την ελευθερία του Tύπου από την αγωγή Δημητριάδη
Scraping article 44: ΣΥΡΙΖΑ – Νίκος Παππάς: 4 απαντήσεις στην κυβέρνηση για την οικονομία – Ας συναντηθεί με την αλήθεια
Scraping article 45: ΠαΣοΚ: Οι αιχμές Δούκα για τις επιλογές Ανδρουλάκη – Τι αναφέρουν κύκλοι του Δημάρχου Αθηναίων
Scraping article 46: Κασσελάκης: Τα νέα γραφεία του στον Ταύρο
Scraping article 47: Ελάχιστα ρεαλιστική πολιτικά η ενίσχυση
Scraping article 48: ΠαΣοΚ: Οι αλλαγές Ανδρουλάκη – Οι ρόλοι σε Δούκα, Γερουλάνο, Διαμαντοπούλου
Scraping article 49: ΣΥΡΙΖΑ: Ο Φάμελλος ζητά η παράσταση «18/9» για τον Π. Φύσσα να μπει στα σχολεία
Scraping article 50: ΣΥΡΙΖΑ: Ο αλγόριθμος του χάους – Οι ημερομηνίες κλειδιά μέχρι το Συνέδριο
Scraping article 51: Ελληνοτουρκικά: «Γεράκια» και «περιστέρια» στο Αιγαίο – Τα ήρεμα νερά και τα υπόγεια ρεύματα
Scraping article 52: Νέα Αριστερά:  Η ιδρυτική πράξη, η αυτοκριτική, η κριτική σε ΣΥΡΙΖΑ – Κασσελάκη
Processing page: https://www.tovima.gr/category/politics/page/4/
Scraping article 53: ΣΥΡΙΖΑ: Οι εκτιμήσεις για τους βουλευτές που στηρίζουν τον Στέφανο Κασσελάκη
Scraping article 54: ΠαΣοΚ: Τα ονόματα που θα ανακοινώσει ο Νίκος Ανδρουλάκης και οι θέσεις που θα πάρουν
Scraping article 55: «Σετ ανακοινώσεων» για πρόσωπα και νέους ρόλους στο ΠαΣοΚ
Scraping article 56: Η υψηλή τέχνη της σάτιρας
Scraping article 57: Φώφη Γεννηματά: Μνημόσυνο για τα 3 χρόνια από τον θάνατό της [Βίντεο, Εικόνες]
Scraping article 58: Κασσελάκης: Νοίκιασε γραφεία για την εκστρατεία του – «Κάποιοι βιάστηκαν να με διώξουν νύχτα»
Scraping article 59: Βάσω Παπανδρέου: Μαχητική, χαρισματική, ανεξάρτητη
Scraping article 60: Παντρεύτηκαν Δούκας και Πολυτάνου – Ποιοι βρέθηκαν στο γάμο (ΦΩΤΟΓΡΑΦΙΕΣ ΚΑΙ ΒΙΝΤΕΟ)
Scraping article 61: Η ομιλία του Μάξιμου Χαρακόπουλου στο επιστημονικό συνέδριο για τον Μικρασιατικό Ελληνισμό
Scraping article 62: ΣΥΡΙΖΑ: Εξ αποστάσεως καρφιά Κασσελάκη – Φάμελλου – Φαραντούρη
Scraping article 63: Δουδωνής σε Σκέρτσο: «Ανεύθυνη είναι η επιλεκτική χρήση των στατιστικών»
Scraping article 64: ΠαΣοΚ: Τηλεφώνημα Ανδρουλάκη σε Διαμαντοπούλου, το ραντεβού με Γερουλάνο
Scraping article 65: Εκλογές ΣΥΡΙΖΑ: Κόμμα διασπασμένο στην κορυφή αλλά και στη βάση
Scraping article 66: Φρέντης Μπελέρης – Σε πρώτο Ενικό: «Όταν έφτασα στην Ελλάδα ήταν σαν να ξαναγεννήθηκα»
Scraping article 67: Νέα Δημοκρατία: Τι απέδωσε το γαλάζιο μασάζ – Πόσο άλλαξε το κλίμα στη ΚΟ
Scraping article 68: ΣΥΡΙΖΑ: Μέλη της ΚΕ παραπέμπουν στην Επιτροπή Δεοντολογίας Γεροβασίλη, Ζαχαριάδη και Γκλέτσο
Scraping article 69: ΣΥΡΙΖΑ: Φιλαλήθης η κυβέρνηση του μπαζώματος;
Scraping article 70: ΣΥΡΙΖΑ: «Καταφανώς ψευδεπίγραφο το φιλελεύθερο προφίλ Μητσοτάκη»
Scraping article 71: Ο άγνωστος καυγάς της Ζωής Κωνσταντοπούλου με τη Ντόρα Μπακογιάννη
Scraping article 72: Οικονόμου: Εμπάθεια από στελέχη κατά Κασσελάκη
Processing page: https://www.tovima.gr/category/politics/page/5/
Scraping article 73: Εκλογές ΣΥΡΙΖΑ: Απορίες Κασσελάκη, αιχμές Φάμελλου – Πολάκη, «αποστάτες» και «μπουζουξίδικο»
Scraping article 74: Νίκος Ανδρουλάκης: «18 Οκτωβρίου 1981, ραντεβού με την Ιστορία»
Scraping article 75: Κασσελάκης: «Συνέδριο σε κλειστό νυχτερινό κέντρο» – Τα 4 ερωτήματα
Scraping article 76: Ο Ανδρουλάκης σχηματίζει σκιώδη κυβέρνηση – Τα πρόσωπα-κλειδιά
Scraping article 77: Μητσοτάκης: Πού ήταν οι υπερπατριώτες όταν προστατεύσαμε τα σύνορα στον Έβρο;
Scraping article 78: Κασσελάκης: «Είναι δυνατόν να βλέπουμε ακόμα φασισμό στην χώρα μας και αντιδημοκρατικές πρακτικές;»
Scraping article 79: Δημοσκόπηση GPO: Πόσο ανέβηκε το ΠαΣοΚ μετά τη νίκη Ανδρουλάκη – Υποχωρεί ο ΣΥΡΙΖΑ
Scraping article 80: ΣΥΡΙΖΑ: Καταδικάζεται, δεν παραπέμπεται ο Κασσελάκης – Σκληρά λόγια στην ΠΓ
Scraping article 81: Γιώργος Παπανδρέου για Βάσω Παπανδρέου: «Άφησε παντού το ιδιαίτερο, βαθύ αποτύπωμά της»
Scraping article 82: ΠαΣοΚ: Η τροπολογία που κατέθεσε για τους Σπαρτιάτες – Η αλλαγή που ζητά
Scraping article 83: Βάσω Παπανδρέου: Η άλλη πλευρά της Σιδηράς Κυρίας – «Η Βάσω συμφώνησε;»
Scraping article 84: Άγρια κόντρα Σπαρτιατών-ΠαΣοΚ: «Έχετε κατακλέψει τον κόσμο» – «Είστε η συνέχεια της ΧΑ»
Scraping article 85: ΣΥΡΙΖΑ: Εμφυλιοπολεμικό το κλίμα στην Πολιτική Γραμματεία – Σκληρά λόγια και διαφωνίες
Scraping article 86: Μαρινάκης για πρωτοβουλία ΠαΣοΚ για Σπαρτιάτες: Μας έχει μπερδέψει
Scraping article 87: Όταν η Βάσω Παπανδρέου πήρε το όπλο της στην κρίσιμη στιγμή για το ΠαΣοΚ
Scraping article 88: Κυριάκος Μητσοτάκης για Βάσω Παπανδρέου: «Άφησε το δικό της αποτύπωμα»
Scraping article 89: Βάσω Παπανδρέου – Το «ναι» ήταν «ναι» και το «όχι» ήταν «όχι»
Scraping article 90: Ανδρουλάκης για Βάσω Παπανδρέου: «Υπηρέτησε χωρίς συμβιβασμούς τις πολιτικές θέσεις της»
Scraping article 91: Η Βάσω Παπανδρέου μέσα από εικόνες της εποχής ΠαΣοΚ
Scraping article 92: Σημίτης για Βάσω Παπανδρέου: «Ήταν μια άξια γυναίκα, τίμησε τις θέσεις που ανέλαβε»
Processing page: https://www.tovima.gr/category/politics/page/6/
Scraping article 93: Νέο «χτύπημα» Κακλαμάνη για το Χρηματιστήριο Ενέργειας
Scraping article 94: Ο πολιτικός κόσμος αποχαιρετά τη Βάσω Παπανδρέου – Η κίνηση Τασούλα
Scraping article 95: Καυγάς Πολάκη με Γεωργιάδη στη Βουλή – Ήρθη η ασυλία του βουλευτή του ΣΥΡΙΖΑ
Scraping article 96: Πέθανε η Βάσω Παπανδρέου
Scraping article 97: ΣΥΡΙΖΑ – Κασσελάκης: Οριακή η κατάσταση πριν από την ΠΓ – Βουλευτές στην έξοδο
Scraping article 98: Μητσοτάκης για μεταναστευτικό: Λείπει μία αποτελεσματική πολιτική επιστροφών από την ΕΕ
Scraping article 99: Γιατί ο Κασιδιάρης βιάζεται να γίνει η δίκη των Σπαρτιατών
Scraping article 100: Τι κρύβει η κόντρα της Λατινοπούλου με την κυβέρνηση
Finished scraping 100 articles
'''